In [1]:
import pytraj as pt
import nglview as nv

_ColormakerRegistry()

In [2]:
import numpy as np
import pandas as pd
import altair as alt

alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [3]:
traj_healthy = pt.load('../ciIP/step7.1b_production.dcd', '../ciIP/step5_assembly.psf')
traj_mutant = pt.load('../Mutant/step7.1b_production.dcd', '../Mutant/step5_assembly.psf')

In [4]:
traj_healthy.xyz.shape, traj_mutant.xyz.shape

((187, 267821, 3), (363, 251836, 3))

In [5]:
# nv.show_pytraj(traj)

In [6]:
from collections import defaultdict

def residue_positions(traj):
    residue_positions = defaultdict(list)

    for index, residue in enumerate(list(traj.top.residues)):
        key = residue.name + str(residue.index)
        residue_positions[key].append(str(index + 1))
        
    return residue_positions

residue_positions_healthy = residue_positions(traj_healthy)
residue_positions_mutant = residue_positions(traj_mutant)

In [7]:
# Cal restar 1 i posar en majúscules
# Per exemple Gly209 -> GLY208
# A més, cal canviar His -> HSD

interesting_residues = [
    'ARG377', 'SER381', 'THR380', 'ARG301', 'SER206', 'GLY208',
    'LYS209', 'GLU210', 'GLY452', 'LEU453', 'PHE454', 'ILE455',
    'TYR557', 'GLY168', 'ILE169', 'GLU267', 'GLY166', 'SER167',
    'ALA164', 'HSD560', 'ARG346', 'ASN564', 'TRP342', 'ARG344',
    'LYS345', 'LYS353', 'LYS448', 'TYR566', 'LYS158', 'LYS173',
    'PHE254', 'GLY259', 'PHE263', 'GLU266', 'SER519', 'THR517',
    'LEU527', 'ALA256', 'ILE258', 'GLY260', 'LEU262', 'LEU531',
    'MET534', 'LEU537', 'MET538', 'VAL522'
]

interesting_common_residues = [r for r in interesting_residues if r not in ['LEU527', 'LEU531', 'MET534', 'VAL522']]

def get_indices(residue_positions, interesting_residues):
    s = ':'
    for index, residue in enumerate(interesting_residues):
        if len(residue_positions[residue]) == 0:
            print(f'Residue {residue} not found in traj')
        else:
            if index != 0: s += ','
            for index2, position in enumerate(residue_positions[residue]):
                if index2 != 0: s += ','
                s += position
    return s[:-1]

get_indices(residue_positions_mutant, interesting_common_residues)

':320,995,324,999,323,998,244,919,149,824,151,826,152,827,153,828,395,1070,396,1071,397,1072,398,1073,499,1174,111,786,112,787,210,885,109,784,110,785,107,782,502,1177,289,964,506,1181,285,960,287,962,288,963,296,971,391,1066,508,1183,101,776,116,791,197,872,202,877,206,881,209,884,462,1137,460,1135,199,874,201,876,203,878,205,880,479,1154,480,115'

In [8]:
get_indices(residue_positions_healthy, interesting_common_residues)

':320,996,324,1000,323,999,244,920,149,825,151,827,152,828,153,829,395,1071,396,1072,397,1073,398,1074,500,1176,111,787,112,788,210,886,109,785,110,786,107,783,503,1179,289,965,507,1183,285,961,287,963,288,964,296,972,391,1067,509,1185,101,777,116,792,197,873,202,878,206,882,209,885,462,1138,460,1136,199,875,201,877,203,879,205,881,480,1156,481,115'

## SAVE DATA

In [10]:
data_healthy_ca = traj_healthy[get_indices(residue_positions_healthy, interesting_common_residues)]['@CA'].xyz
n_frames = data_healthy_ca.shape[0]
data_healthy_ca.shape

(187, 84, 3)

In [11]:
data_mutant_ca = traj_mutant[get_indices(residue_positions_mutant, interesting_common_residues)]['@CA'].xyz[:n_frames, :, :]
data_mutant_ca.shape

(187, 84, 3)

In [ ]:
DATA_DIR = "data/"
# healthy patient
np.save(DATA_DIR+"data_healthy_ca.npy", data_healthy_ca)
# mutant patient
np.save(DATA_DIR+"data_mutant_ca.npy", data_healthy_ca)